In [43]:
import polars as pl
import numpy as np

In [44]:
df = pl.read_csv("data/distress.csv",infer_schema_length=10000).with_columns([
    pl.col("Company").cast(pl.Utf8).cast(pl.Categorical),
    # pl.col("Time").cast(pl.Utf8).cast(pl.Categorical),
])

In [45]:
df = df.groupby("Company").apply(lambda d: d.sort(by="Time", descending=True).head(1)).drop("Time")
df.head()

Company,Financial Distress,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,…,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
cat,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,i64
"""113""",-1.4236,1.6988,0.028931,0.56967,0.28909,-0.036253,0.21861,0.67484,0.10007,0.067229,0.0044898,0.53145,-6.8274,0.43033,1.3238,1.6518,-0.1254,0.91888,0.014097,0.018368,0.045062,0.54398,0.65661,-0.008192,0.31284,135.57,12.384,1.3224,0.54136,0.25683,0.21111,3.7495,0.73026,0.54916,0.91142,217.94,…,1.326,2016.5,0.72696,1.0082,11.143,-0.090897,-0.002368,62.98,-0.55287,0.31719,0.67373,0.0017909,0.0061949,0.0,7.4166,7.105,14.321,18.77,124.76,26.124,11.8,8322.8,0.1896,15.6,24.579,0.0,36.0,107.09,31.31,30.194,17.0,16.0,0.4,20,0.021039,10,43
"""417""",0.041872,29.233,0.043577,0.034891,0.13391,-0.028079,0.92955,28.793,0.32542,0.045153,0.077611,0.96247,-10251.0,0.96511,0.036153,1.84,-0.20968,10.352,0.19638,0.0,5.9646,0.13913,3.8705,0.0058501,0.032925,49.716,12.808,0.14406,0.45551,0.0019667,0.948,40.637,7.0793,0.94363,10.413,195.65,…,0.036153,1101.1,0.034115,0.99415,10.797,-0.18618,0.0007834,6984.0,0.47901,0.01286,-0.080178,0.0,0.0,0.0,-1.5863,-7.5046,12.145,-20.229,28.286,30.023,11.0,34501.0,-32.209,11.9,13.871,1.0,29.0,91.7,227.5,214.5,21.0,20.5,8.6,33,0.088544,5,5
"""193""",0.0023833,1.0011,0.02073,0.67372,1.5659,0.035741,0.000631,0.73972,0.013238,0.063533,0.0072724,0.59495,0.72375,0.32628,2.0648,4.4555,0.022825,12.006,0.061082,0.031536,0.10278,2.632,5.8663,0.10944,0.59432,67.367,12.181,2482.3,-0.13094,0.079401,0.43963,11.741,0.28075,0.88214,13.481,80.798,…,2.0648,1060.3,1.8215,0.89056,12.629,0.027108,0.17137,3214.0,0.9414,0.11615,-0.003179,-0.04465,-0.028514,0.0,-0.38605,-5.2355,10.898,-16.307,15.999,28.101,10.9,34076.0,-38.688,12.825,16.046,1.25,29.25,92.0,221.42,212.0,21.25,20.875,8.05,26,0.023633,39,50
"""129""",0.0046932,2.3099,-0.050217,0.41612,0.48674,-0.011442,0.49129,0.76675,-0.10317,-0.086006,0.15987,0.86635,-0.22657,0.58388,0.71269,1.9625,-0.023507,0.76784,0.033905,0.10375,0.090399,0.56183,3.8558,0.10273,0.37506,-124.68,12.295,0.99074,-0.1185,0.041063,0.28758,7.4849,0.59082,0.90132,0.85575,183.44,…,0.71393,1449.7,0.64236,0.89727,11.574,0.026422,0.050002,681.73,-1.0,0.56879,-0.12712,0.000389,0.000799,0.0,-1.5863,-7.5046,12.145,-20.229,28.286,30.023,11.0,34501.0,-32.209,11.9,13.871,1.0,29.0,91.7,227.5,214.5,21.0,20.5,8.6,9,0.39694,25,54
"""289""",0.53921,1.0377,0.02058,0.92838,1.1883,0.059205,0.034824,0.81537,0.017318,0.28734,0.022124,0.95944,1.79,0.07162,12.963,1.6059,0.049823,5.2168,0.013942,0.027835,0.015078,1.2385,33.184,0.10502,0.92462,508.34,16.106,34.124,0.26317,0.0037594,0.7539,1.5521,0.63443,0.99595,5.8289,224.17,…,13.527,1769.1,12.91,0.89498,16.279,0.052669,0.12479,8153.6,-0.1454,0.20387,0.16196,-0.006999,-0.00589,0.23237,-1.5863,-7.5046,12.145,-20.229,28.286,30.023,11.0,34501.0,-32.209,11.9,13.871,1.0,29.0,91.7,227.5,214.5,21.0,20.5,8.6,29,0.54649,4,42


In [46]:
df.describe()

describe,Company,Financial Distress,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,…,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""422""",422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,…,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0,422.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,0.32877,1.406708,0.061094,0.64012,0.794028,0.081276,0.095781,0.84946,495.485553,0.103794,0.096211,0.657577,131574.975556,0.35988,3.327774,5704.76248,-56.76107,2222.370987,0.033123,329.480207,0.101732,1.271689,20.590531,0.205219,0.561797,536.422155,13.840571,-674.881804,0.129983,0.078324,0.377503,454232.439464,347.36402,0.881374,2613.9681,…,10288.220491,2208.889123,2.923207,0.79478,13.302714,-51.677523,0.142184,4729.247614,-0.23815,0.239146,4.709018,-0.012721,-18.683917,0.079031,0.481799,-2.668214,8.46447,-11.081203,27.480805,28.610945,11.149775,27138.220142,-10.94026,13.956813,18.082436,1.349603,29.785592,90.580273,173.721197,165.149974,19.832448,18.684893,4.728085,19.900474,0.86368,14.672986,34.509479
"""std""",null,2.162473,1.594928,0.108615,0.20522,0.610424,0.112206,0.232494,1.511292,10173.949997,0.385378,0.147456,0.212054,1.3736e6,0.20522,3.732249,65730.159349,1167.526932,23505.610509,0.039578,6766.409436,0.388214,0.979705,152.40995,0.176835,0.199377,1186.767568,1.767857,14020.631575,0.300926,0.097413,0.187029,9.1515e6,7107.106362,0.132942,28409.32662,…,122909.429247,2072.994335,3.345164,0.176834,2.017767,1063.9395,0.12431,11437.128045,0.546398,0.159512,64.981021,0.025926,383.20155,0.101569,3.179061,6.77484,6.523011,12.787869,29.611517,3.747765,0.636375,11178.639118,71.047061,4.345468,11.475312,0.97479,2.44308,9.911088,78.556797,74.734959,2.062139,3.143683,6.587361,7.629456,1.688903,10.14279,14.565232
"""min""",null,-8.6317,0.20651,-0.25808,0.034891,5.3500e-7,-0.26844,-0.59924,0.039691,-1.0365,-3.6112,-0.31866,0.055894,-161000.0,0.032101,0.036153,0.0061223,-23984.0,0.0000869,0.000305,0.0,0.000629,0.000006,8.3000e-7,-0.28147,0.032925,-3374.6,9.6165,-288000.0,-0.49944,0.0,0.023358,0.0069371,0.015421,0.10735,0.0000869,…,0.036153,533.07,0.034115,0.0060043,0.0,-21856.0,-0.12254,0.03125,-1.0,0.0,-0.99779,-0.21617,-7872.0,-0.34906,-7.714,-11.824,-25.736,-21.411,-21.907,15.916,10.3,7958.3,-58.122,10.4,-10.665,0.0,22.0,54.806,25.235,24.551,15.5,12.0,-20.2,1.0,-0.49922,1.0,5.0
"""max""",null,20.176,29.233,0.51456,0.9679,4.8973,0.4787,0.92955,28.793,209000.0,1.7823,0.54987,0.99827,2.6e7,0.96511,30.152,1.03e6,3.7147,334000.0,0.32354,139000.0,5.9646,7.5295,2846.3,0.994,0.93563,9851.6,19.106,2482.3,4.6254,0.69046,0.948,1.88e8,146000.0,1.0,451000.0,…,1.66e6,19207.0,26.398,1.2815,19.08,3.7244,0.83147,121000.0,1.0,0.92653,1182.8,0.12164,1.4029,0.86332,7.4166,15.381,14.321,18.77,124.76,39.432,13.5,34501.0,364.95,34.7,61.476,4.0,36.5,120.87,227.5,214.5,22.0,22.0,8.6,37.0,18.107,49.0,74.0
"""median""",null,0.195765,1.138,0.0412535,0.67835,0.66902,0.061591,0.088341,0.64554

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error

In [70]:
y = df["Financial Distress"]
X = df.drop(["Financial Distress", "Company"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = LinearRegression(fit_intercept=True)

reg.fit(X_train.to_numpy(), y_train.to_numpy())

y_pred = reg.predict(X_test.to_numpy())

print(f"R^2: {reg.score(X_test.to_numpy(), y_test.to_numpy())}")
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

R^2: -11.083799788456615
MSE: 36.144027365997886


In [71]:
import xgboost as xgb

In [72]:
model = xgb.XGBRegressor()
model.fit(X_train.to_numpy(), y_train.to_numpy())
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_test.to_numpy(), y_test.to_numpy(), scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('MSE: %.3f (%.3f)' % (scores.mean(), scores.std()) )


MSE: 1.269 (0.620)
